In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/stsb-xlm-r-multilingual')
model = AutoModel.from_pretrained('sentence-transformers/stsb-xlm-r-multilingual')

# Function to compute embeddings
def compute_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embeddings

# Sample dataset
documents = [
    "ප්‍රත්‍යවර්තක ධාරාවක්‌ සඳහා සමීකරණය 1 ~ 1, ල $” ලෙස ලිවිය හැකිය. මෙහි ! යනු කාලය වන අතර 1",
    "ශුන්‍ය ලක්ෂ්‍ය ශක්තිය යනු ක්වොන්ටම් යාන්ත්‍රික භෞතික පද්ධතියකට තිබිය හැකි අවම ශක්තියයි. “පහත සඳහන් අය අතරින් මෙම සංකල්පය ඉදිරිපත් කළේ කවුද?",
    "Night Vision උපාංගවල බහුලව භාවිතා වන තරංග අතරින් පහත සඳහන් තරංග මොනවාද?",
    "පහත සඳහන් දේවලින් දර්පණ සූත්‍රය කුමක්ද?[A] v + u =f [B] 1/v + 1/u = 1/f [C] 1/v + 1/u = f [D] ඉහත කිසිවක් නොවේ",
    "වස්තුව අනන්තයේ තැබූ විට උත්තල දර්පණයකින් සෑදෙන රූපයේ ස්වභාවය කුමක්ද?",
    "ගැල්වනික් සෛලයක ඉලෙක්ට්‍රෝලය හරහා ධාරාව ගමන් කරන දිශාව කුමක්ද?",
    "සන්නායක සම්බන්ධයෙන් පහත සඳහන් දේවලින් නිවැරදි වන්නේ කුමක්ද?",
    "මෝටර් රථ රියදුරන් පිටුපස මාර්ගය බැලීමට භාවිතා කරන්නේ කුමන ආකාරයේ දර්පණයක්ද?",
    "සෙල්සියස් සහ ෆැරන්හයිට් පරිමාණයන් මත සංඛ්‍යාත්මක අගයන් සමාන වන්නේ කුමන උෂ්ණත්වයකදීද (සෙල්සියස් අංශකයෙන්),?",

    "What is the rate of flow of electric charges called?",
    " Which of the following is the SI Unit of Electric Current?",
    "When one unit electric charge moves from one point to another point in an electric circuit, then the amount of work done in joules is known as",
    "The hindrance presented by material of conductor to the smooth passing of electric current is known as",
    "The resistance of a conductor depends on",
    "Which of the following does not produce a sound wave?",
    "Which of the following is the unit for frequency",
    "he amplitude of a wave is the distance between two consecutive",
    "Three rays of light are incident on the boundary between a glass block and air."
]

# Compute embeddings for the dataset
embeddings = compute_embeddings(documents)

# Initialize FAISS index
d = embeddings.shape[1]  # dimension of the embeddings
index = faiss.IndexFlatL2(d)  # using L2 distance

# Add embeddings to the index
index.add(embeddings)


print("Number of documents in the index:", index.ntotal)

Number of documents in the index: 18


In [4]:

query = "What is the rate of flow of electric charges called?"

query_embedding = compute_embeddings([query]).reshape(1, -1)

k = 20
distances, indices = index.search(query_embedding, k)

for i, idx in enumerate(indices[0]):
    if distances[0][i] < 100:
        print(f"Document {idx}: {documents[idx]} (Distance: {distances[0][i]})")


Document 9: What is the rate of flow of electric charges called? (Distance: 3.221632682848252e-11)
Document 10:  Which of the following is the SI Unit of Electric Current? (Distance: 56.75519561767578)
Document 11: When one unit electric charge moves from one point to another point in an electric circuit, then the amount of work done in joules is known as (Distance: 77.05288696289062)
Document 5: ගැල්වනික් සෛලයක ඉලෙක්ට්‍රෝලය හරහා ධාරාව ගමන් කරන දිශාව කුමක්ද? (Distance: 90.00004577636719)
